# EDA (Explotary Data Analysis)

In [1]:
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
# Import necessary libraries
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Embedding, LSTM, Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification ,TFBertForSequenceClassification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:
data=pd.read_csv("/content/adress_dataset.csv")

In [5]:
data.head(10)

,Address,Target
0,فاقوس,Not Cairo governorate
1,العمري,Cairo governorate
2,محمد مظهر,Cairo governorate
3,المعادى,Cairo governorate
4,حدائق القبة,Cairo governorate
5,كفر صقر,Not Cairo governorate
6,مَغَاغَة,Not Cairo governorate
7,الكردي,Cairo governorate
8,الفيوم,Not Cairo governorate
9,قَلْيوب,Not Cairo governorate


In [6]:
data.duplicated().sum()

0

In [7]:
len(data)

265

In [8]:
target_counts = data['Target'].value_counts()

print(target_counts)


Not Cairo governorate    144
Cairo governorate        121
Name: Target, dtype: int64


In [9]:
data=pd.get_dummies(data, columns=['Target'])

In [10]:
data.head()

,Address,Target_Cairo governorate,Target_Not Cairo governorate
0,فاقوس,0,1
1,العمري,1,0
2,محمد مظهر,1,0
3,المعادى,1,0
4,حدائق القبة,1,0


In [11]:
data.drop(['Target_Not Cairo governorate'], axis=1, inplace =True )

In [12]:
data.head()

,Address,Target_Cairo governorate
0,فاقوس,0
1,العمري,1
2,محمد مظهر,1
3,المعادى,1
4,حدائق القبة,1


In [13]:
data.rename(columns={'Target_Cairo governorate': 'is_Cairo governorate'}, inplace=True)

In [14]:
data.head()

,Address,is_Cairo governorate
0,فاقوس,0
1,العمري,1
2,محمد مظهر,1
3,المعادى,1
4,حدائق القبة,1


# Building Simple Architecture

In [15]:
# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['Address'].astype(str).values,
    data['is_Cairo governorate'].values,
    test_size=0.2,
    random_state=42
)


In [16]:
# Vectorize the addresses
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()
feature_names = vectorizer.get_feature_names_out()

In [17]:
train_labels = train_labels.astype(float)
test_labels = test_labels.astype(float)

In [18]:
# Create a simple neural network model
model = Sequential([
    Embedding(input_dim=len(feature_names), output_dim=50, input_length=X_train.shape[1]),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [19]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
# Train the model
model.fit(X_train, train_labels, epochs=20, batch_size=16, validation_split=0.1)


Epoch 1/20
12/12 [==============================] - 4s 60ms/step - loss: 0.6936 - accuracy: 0.5105 - val_loss: 0.6961 - val_accuracy: 0.3182
Epoch 2/20
12/12 [==============================] - 0s 14ms/step - loss: 0.6907 - accuracy: 0.5842 - val_loss: 0.7030 - val_accuracy: 0.3182
Epoch 3/20
12/12 [==============================] - 0s 18ms/step - loss: 0.6884 - accuracy: 0.5842 - val_loss: 0.7094 - val_accuracy: 0.3182
Epoch 4/20
12/12 [==============================] - 0s 12ms/step - loss: 0.6862 - accuracy: 0.5842 - val_loss: 0.7152 - val_accuracy: 0.3182
Epoch 5/20
12/12 [==============================] - 0s 14ms/step - loss: 0.6851 - accuracy: 0.5842 - val_loss: 0.7229 - val_accuracy: 0.3182
Epoch 6/20
12/12 [==============================] - 0s 30ms/step - loss: 0.6831 - accuracy: 0.5842 - val_loss: 0.7289 - val_accuracy: 0.3182
Epoch 7/20
12/12 [==============================] - 0s 23ms/step - loss: 0.6820 - accuracy: 0.5842 - val_loss: 0.7422 - val_accuracy: 0.3182
Epoch 8/20
12

In [21]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, test_labels)
print(f'Test Accuracy: {accuracy}')


2/2 [==============================] - 0s 11ms/step - loss: 0.7119 - accuracy: 0.4906
Test Accuracy: 0.49056604504585266


Conclusion :
the accuracy is bad due to the model is too simple and we should make it more complex

Approches to make the model be more complex :     

- Word Embeddings:

Instead of a basic bag-of-words approach, use pre-trained word embeddings

* Recurrent Neural Network (RNN) or Long Short-Term Memory (LSTM):

Utilize RNNs or LSTMs to capture sequential dependencies in the address text.

* Bidirectional RNN or LSTM:

Allow the model to learn from both past and future context


# LSTM

In [22]:
model = Sequential([
    Embedding(input_dim=len(feature_names), output_dim=100, input_length=X_train.shape[1]),
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [23]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
# Train the model
model.fit(X_train, train_labels, epochs=10, batch_size=64, validation_split=0.1)


Epoch 1/10
3/3 [==============================] - 8s 828ms/step - loss: 0.6903 - accuracy: 0.5842 - val_loss: 0.7199 - val_accuracy: 0.3182
Epoch 2/10
3/3 [==============================] - 0s 66ms/step - loss: 0.6838 - accuracy: 0.5842 - val_loss: 0.7325 - val_accuracy: 0.3182
Epoch 3/10
3/3 [==============================] - 0s 63ms/step - loss: 0.6806 - accuracy: 0.5842 - val_loss: 0.7653 - val_accuracy: 0.3182
Epoch 4/10
3/3 [==============================] - 0s 58ms/step - loss: 0.6870 - accuracy: 0.5842 - val_loss: 0.8111 - val_accuracy: 0.3182
Epoch 5/10
3/3 [==============================] - 0s 56ms/step - loss: 0.6812 - accuracy: 0.5842 - val_loss: 0.7873 - val_accuracy: 0.3182
Epoch 6/10
3/3 [==============================] - 0s 51ms/step - loss: 0.6793 - accuracy: 0.5842 - val_loss: 0.7664 - val_accuracy: 0.3182
Epoch 7/10
3/3 [==============================] - 0s 49ms/step - loss: 0.6787 - accuracy: 0.5842 - val_loss: 0.7564 - val_accuracy: 0.3182
Epoch 8/10
3/3 [==========

In [25]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, test_labels)
print(f'Test Accuracy: {accuracy}')

2/2 [==============================] - 0s 18ms/step - loss: 0.7041 - accuracy: 0.4906
Test Accuracy: 0.49056604504585266


Conclusion :     
the training accuracy became higher but test accuracy still low

# Bidirectional LSTM-based

In [26]:
# Tokenize the addresses
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)

In [27]:
# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(train_texts)
X_test_sequences = tokenizer.texts_to_sequences(test_texts)

In [28]:
# Pad sequences to ensure consistent length
X_train_padded = pad_sequences(X_train_sequences)
X_test_padded = pad_sequences(X_test_sequences, maxlen=X_train_padded.shape[1])

In [29]:
# Create a Bidirectional LSTM-based
model_b = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_train_padded.shape[1]),
    Bidirectional(LSTM(50, return_sequences=True)),
    Bidirectional(LSTM(50)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [30]:
# Compile the model
model_b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
# Train the model
model_b.fit(X_train_padded, train_labels, epochs=10, batch_size=64, validation_split=0.1)

Epoch 1/10
3/3 [==============================] - 9s 1s/step - loss: 0.6920 - accuracy: 0.5211 - val_loss: 0.6981 - val_accuracy: 0.3182
Epoch 2/10
3/3 [==============================] - 1s 175ms/step - loss: 0.6882 - accuracy: 0.5842 - val_loss: 0.7041 - val_accuracy: 0.3182
Epoch 3/10
3/3 [==============================] - 0s 111ms/step - loss: 0.6842 - accuracy: 0.5842 - val_loss: 0.7114 - val_accuracy: 0.3182
Epoch 4/10
3/3 [==============================] - 0s 164ms/step - loss: 0.6780 - accuracy: 0.5842 - val_loss: 0.7216 - val_accuracy: 0.3182
Epoch 5/10
3/3 [==============================] - 1s 266ms/step - loss: 0.6693 - accuracy: 0.5842 - val_loss: 0.7359 - val_accuracy: 0.3182
Epoch 6/10
3/3 [==============================] - 0s 42ms/step - loss: 0.6558 - accuracy: 0.6053 - val_loss: 0.7543 - val_accuracy: 0.3182
Epoch 7/10
3/3 [==============================] - 1s 265ms/step - loss: 0.6376 - accuracy: 0.6211 - val_loss: 0.7809 - val_accuracy: 0.3182
Epoch 8/10
3/3 [========

In [32]:
# Evaluate the model on the test set
loss, accuracy = model_b.evaluate(X_test_padded, test_labels)
print(f'Test Accuracy: {accuracy}')

2/2 [==============================] - 0s 14ms/step - loss: 0.8505 - accuracy: 0.5283
Test Accuracy: 0.5283018946647644


Conclusion :      
Bidirectional LSTM-based gives higher accuracy in training and testing than traditional LSTM


# Pretrained (BERT)

In [33]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=1)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
train_tokens = tokenizer(train_texts.tolist(), padding=True, truncation=True, return_tensors='tf')
test_tokens = tokenizer(test_texts.tolist(), padding=True, truncation=True, return_tensors='tf')

In [35]:
# Prepare labels
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [36]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

# Fine-tune the BERT model for address verification
optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
6/6 [==============================] - 60s 2s/step - loss: 4.9650 - accuracy: 0.5211 - val_loss: 4.8520 - val_accuracy: 0.6818
Epoch 2/3
6/6 [==============================] - 9s 2s/step - loss: 8.9088 - accuracy: 0.4158 - val_loss: 4.8520 - val_accuracy: 0.6818
Epoch 3/3
6/6 [==============================] - 10s 2s/step - loss: 8.9088 - accuracy: 0.4158 - val_loss: 4.8520 - val_accuracy: 0.6818


In [37]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

2/2 [==============================] - 1s 357ms/step - loss: 7.4808 - accuracy: 0.5094
Test Accuracy: 0.5094339847564697


# Pretrained (DistilBERT)

In [38]:
# Load pre-trained DistilBERT model and tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [39]:
# Tokenize the addresses
train_tokens = tokenizer(train_texts.tolist(), padding=True, truncation=True, return_tensors='tf')
test_tokens = tokenizer(test_texts.tolist(), padding=True, truncation=True, return_tensors='tf')

In [40]:
# Convert BatchEncoding objects to NumPy arrays
X_train = {key: np.array(value) for key, value in train_tokens.items()}
X_test = {key: np.array(value) for key, value in test_tokens.items()}

In [41]:
# Prepare labels
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [42]:
# Fine-tune the DistilBERT model for address verification
optimizer = Adam(lr=2e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3, batch_size=64, validation_split=0.1)

Epoch 1/3
3/3 [==============================] - 24s 969ms/step - loss: 4.7927 - accuracy: 0.5526 - val_loss: 10.5170 - val_accuracy: 0.3182
Epoch 2/3
3/3 [==============================] - 1s 197ms/step - loss: 5.6828 - accuracy: 0.5842 - val_loss: 4.8520 - val_accuracy: 0.6818
Epoch 3/3
3/3 [==============================] - 0s 156ms/step - loss: 8.9088 - accuracy: 0.4158 - val_loss: 4.8520 - val_accuracy: 0.6818


In [43]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

2/2 [==============================] - 2s 42ms/step - loss: 7.4808 - accuracy: 0.5094
Test Accuracy: 0.5094339847564697


Conclusion :     
The best performance is the model of Bidirectional LSTM-based so it will be saved for a deployment

Note :     
to make the accuracy more higher we should change the structure of the data by make the address be a statment instead of one word and scrap more and more data

In [44]:
# Save the model
model_b.save('model_weights.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
